# Customer Purchase Prediction Model - 5 Features

## Farm2Home ML Model Training

This notebook trains a machine learning model to predict customer purchase categories using exactly 5 features:
- `totalOrders`
- `purchaseFrequency`
- `avgOrderValue`
- `lastPurchaseDaysAgo`
- `totalItemsBought`

**Output:** `model.pkl` and `scaler.pkl` (both expecting 5 features)


## 1. Import Libraries


In [11]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
import xgboost as xgb

warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")


✅ Libraries imported successfully


## 2. Load Customer Dataset


In [12]:
# Load data from CSV files
print("📚 Loading datasets...")

try:
    # Try loading from root directory
    customers_df = pd.read_csv('../customers.csv')
    orders_df = pd.read_csv('../orders.csv')
    products_df = pd.read_csv('../products.csv')
    print("✅ Loaded from root directory")
except:
    try:
        # Try loading from current directory
        customers_df = pd.read_csv('customers.csv')
        orders_df = pd.read_csv('orders.csv')
        products_df = pd.read_csv('products.csv')
        print("✅ Loaded from current directory")
    except Exception as e:
        print(f"⚠️  Could not load CSV files: {e}")
        print("📊 Generating synthetic data for training...")
        # Generate synthetic data if CSV files not found
        np.random.seed(42)
        n_customers = 1000
        
        # Create synthetic customers
        customers_df = pd.DataFrame({
            'id': [f'CUST_{i}' for i in range(1, n_customers + 1)],
            'name': [f'Customer {i}' for i in range(1, n_customers + 1)],
            'email': [f'customer{i}@example.com' for i in range(1, n_customers + 1)]
        })
        
        # Create synthetic orders
        orders_list = []
        categories = ['Vegetables', 'Fruits', 'Grains', 'Dairy', 'Spices']
        
        for i in range(1, n_customers + 1):
            n_orders = np.random.randint(1, 50)
            for j in range(n_orders):
                days_ago = np.random.randint(0, 365)
                orders_list.append({
                    'id': f'ORD_{i}_{j}',
                    'customerId': f'CUST_{i}',
                    'totalAmount': np.random.uniform(500, 5000),
                    'createdAt': (datetime.now() - timedelta(days=days_ago)).isoformat(),
                    'category': np.random.choice(categories)
                })
        
        orders_df = pd.DataFrame(orders_list)
        products_df = pd.DataFrame({
            'id': range(1, 101),
            'category': np.random.choice(categories, 100)
        })
        
        print(f"✅ Generated synthetic data: {len(customers_df)} customers, {len(orders_df)} orders")

print(f"\n📊 Dataset Summary:")
print(f"  - Customers: {customers_df.shape}")
print(f"  - Orders: {orders_df.shape}")
print(f"  - Products: {products_df.shape}")


📚 Loading datasets...
✅ Loaded from root directory

📊 Dataset Summary:
  - Customers: (15000, 11)
  - Orders: (92910, 21)
  - Products: (10799, 9)


## 3. Feature Engineering (5 Features Only)


In [13]:
print("🔧 Engineering features from customer order history...")

# Ensure orders have proper date format
if 'createdAt' in orders_df.columns:
    orders_df['createdAt'] = pd.to_datetime(orders_df['createdAt'], errors='coerce')
elif 'created_at' in orders_df.columns:
    orders_df['createdAt'] = pd.to_datetime(orders_df['created_at'], errors='coerce')
elif 'orderDate' in orders_df.columns:
    orders_df['createdAt'] = pd.to_datetime(orders_df['orderDate'], errors='coerce')
else:
    # Create synthetic dates if no date column
    orders_df['createdAt'] = pd.date_range(end=datetime.now(), periods=len(orders_df), freq='D')

# Ensure customerId column exists
if 'customerId' not in orders_df.columns:
    if 'customer_id' in orders_df.columns:
        orders_df['customerId'] = orders_df['customer_id']
    elif 'customerID' in orders_df.columns:
        orders_df['customerId'] = orders_df['customerID']
    else:
        print("⚠️  No customer ID column found, using first available ID column")
        orders_df['customerId'] = orders_df.iloc[:, 0]

# Ensure totalAmount column exists
if 'totalAmount' not in orders_df.columns:
    if 'total_amount' in orders_df.columns:
        orders_df['totalAmount'] = orders_df['total_amount']
    elif 'amount' in orders_df.columns:
        orders_df['totalAmount'] = orders_df['amount']
    else:
        print("⚠️  No amount column found, generating random amounts")
        orders_df['totalAmount'] = np.random.uniform(500, 5000, len(orders_df))

# Calculate features for each customer
features_list = []

# Get customer IDs
if 'id' in customers_df.columns:
    customer_ids = customers_df['id'].unique()
elif 'customerId' in customers_df.columns:
    customer_ids = customers_df['customerId'].unique()
else:
    customer_ids = orders_df['customerId'].unique()

print(f"\n📈 Processing {len(customer_ids)} customers...")

for customer_id in customer_ids:
    customer_orders = orders_df[orders_df['customerId'] == customer_id].copy()
    
    if len(customer_orders) == 0:
        continue
    
    # Sort by date
    customer_orders = customer_orders.sort_values('createdAt')
    
    # Feature 1: totalOrders
    total_orders = len(customer_orders)
    
    # Feature 2: purchaseFrequency (orders per month)
    if total_orders > 1:
        date_range = (customer_orders['createdAt'].max() - customer_orders['createdAt'].min()).days
        if date_range > 0:
            purchase_frequency = (total_orders / date_range) * 30  # orders per month
        else:
            purchase_frequency = total_orders  # if all orders same day
    else:
        purchase_frequency = 0.0
    
    # Feature 3: avgOrderValue
    avg_order_value = customer_orders['totalAmount'].mean()
    
    # Feature 4: lastPurchaseDaysAgo
    last_order_date = pd.Timestamp(customer_orders['createdAt'].max())
    # Normalize to timezone-naive to avoid timezone mismatch errors
    if last_order_date.tz is not None:
        last_order_date = last_order_date.tz_localize(None)
    # Use pd.Timestamp.now() and normalize to timezone-naive
    now = pd.Timestamp.now()
    if now.tz is not None:
        now = now.tz_localize(None)
    days_ago = (now - last_order_date).days
    
    # Feature 5: totalItemsBought
    # Try to get quantity from order items, otherwise estimate
    if 'quantity' in customer_orders.columns:
        total_items = customer_orders['quantity'].sum()
    elif 'itemCount' in customer_orders.columns:
        total_items = customer_orders['itemCount'].sum()
    else:
        # Estimate: assume 2-5 items per order
        total_items = total_orders * np.random.randint(2, 6)
    
    # Get target category - use most frequent category in customer's orders
    if 'category' in customer_orders.columns and customer_orders['category'].notna().any():
        # Get the most frequent category for this customer
        category_counts = customer_orders['category'].value_counts()
        if len(category_counts) > 0:
            target_category = category_counts.index[0]
        else:
            target_category = 'Vegetables'
    else:
        # If no category column, try to get from products
        # Otherwise, assign based on customer behavior patterns
        if avg_order_value > 3000:
            target_category = 'Fruits'
        elif avg_order_value > 2000:
            target_category = 'Vegetables'
        elif purchase_frequency > 2:
            target_category = 'Grains'
        elif total_items > 50:
            target_category = 'Dairy'
        else:
            target_category = 'Spices'
    
    features_list.append({
        'customerId': customer_id,
        'totalOrders': total_orders,
        'purchaseFrequency': purchase_frequency,
        'avgOrderValue': avg_order_value,
        'lastPurchaseDaysAgo': days_ago,
        'totalItemsBought': total_items,
        'targetCategory': target_category
    })

# Create features DataFrame
features_df = pd.DataFrame(features_list)

print(f"\n✅ Feature engineering complete!")
print(f"  - Total records: {len(features_df)}")
print(f"\n📋 Feature Summary:")
print(features_df[['totalOrders', 'purchaseFrequency', 'avgOrderValue', 'lastPurchaseDaysAgo', 'totalItemsBought']].describe())

print(f"\n📊 Target Categories:")
print(features_df['targetCategory'].value_counts())

# Check if we have diversity in target categories
unique_categories = features_df['targetCategory'].nunique()
if unique_categories < 2:
    print(f"\n⚠️  WARNING: Only {unique_categories} unique category found!")
    print("   Creating diverse categories based on customer features...")
    
    # Assign categories based on customer behavior to ensure diversity
    features_df['targetCategory'] = pd.cut(
        features_df['avgOrderValue'],
        bins=5,
        labels=['Spices', 'Grains', 'Vegetables', 'Dairy', 'Fruits']
    ).astype(str)
    
    # Fill any NaN values
    features_df['targetCategory'] = features_df['targetCategory'].fillna('Vegetables')
    
    print(f"\n✅ Created diverse categories:")
    print(features_df['targetCategory'].value_counts())


🔧 Engineering features from customer order history...

📈 Processing 15000 customers...

✅ Feature engineering complete!
  - Total records: 14973

📋 Feature Summary:
        totalOrders  purchaseFrequency  avgOrderValue  lastPurchaseDaysAgo  \
count  14973.000000       14973.000000   14973.000000         14973.000000   
mean       6.205169           1.665165    2745.021970            28.793962   
std        2.490159           1.675048    1001.165524            28.017591   
min        1.000000           0.000000      23.040000             0.000000   
25%        4.000000           1.153846    2074.190000             9.000000   
50%        6.000000           1.500000    2683.597500            20.000000   
75%        8.000000           1.907514    3327.691667            40.000000   
max       18.000000          60.000000    9154.200000           180.000000   

       totalItemsBought  
count      14973.000000  
mean          65.049222  
std           29.556704  
min            1.000000  
25

## 4. Prepare Data for Training


In [14]:
print("🎯 Preparing data for model training...")

# Define the EXACT 5 features we will use
FEATURE_COLUMNS = [
    'totalOrders',
    'purchaseFrequency',
    'avgOrderValue',
    'lastPurchaseDaysAgo',
    'totalItemsBought'
]

# Extract features and target
X = features_df[FEATURE_COLUMNS].copy()
y = features_df['targetCategory'].copy()

# Handle missing values
X = X.fillna(0)

# Remove any infinite values
X = X.replace([np.inf, -np.inf], 0)

print(f"\n✅ Data prepared:")
print(f"  - Features shape: {X.shape}")
print(f"  - Target shape: {y.shape}")
print(f"  - Feature columns: {FEATURE_COLUMNS}")

# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print(f"\n✅ Target encoded: {len(label_encoder.classes_)} categories")
print(f"  Categories: {list(label_encoder.classes_)}")

# Check if we have enough classes for classification
n_unique_classes = len(np.unique(y_encoded))
if n_unique_classes < 2:
    raise ValueError(f"ERROR: Need at least 2 classes for classification, but found only {n_unique_classes} class(es). "
                     f"Please check your data - all customers may have the same target category.")

# Split data - use stratify only if we have enough samples per class
try:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
    )
except ValueError as e:
    # If stratify fails (not enough samples per class), split without stratify
    print(f"⚠️  Stratified split failed: {e}")
    print("   Using non-stratified split instead...")
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_encoded, test_size=0.2, random_state=42
    )

print(f"\n✅ Data split:")
print(f"  - Training: {X_train.shape[0]} samples")
print(f"  - Testing: {X_test.shape[0]} samples")


🎯 Preparing data for model training...

✅ Data prepared:
  - Features shape: (14973, 5)
  - Target shape: (14973,)
  - Feature columns: ['totalOrders', 'purchaseFrequency', 'avgOrderValue', 'lastPurchaseDaysAgo', 'totalItemsBought']

✅ Target encoded: 5 categories
  Categories: ['Dairy', 'Fruits', 'Grains', 'Spices', 'Vegetables']

✅ Data split:
  - Training: 11978 samples
  - Testing: 2995 samples


## 5. Scale Features


In [15]:
print("📏 Scaling features with StandardScaler...")

# Create and fit scaler on training data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"✅ Features scaled:")
print(f"  - Training shape: {X_train_scaled.shape}")
print(f"  - Test shape: {X_test_scaled.shape}")
print(f"  - Scaler expects {scaler.n_features_in_} features")

# Verify feature count
assert scaler.n_features_in_ == 5, f"ERROR: Scaler expects {scaler.n_features_in_} features, but should be 5!"
print("\n✅ VERIFIED: Scaler correctly expects exactly 5 features")


📏 Scaling features with StandardScaler...
✅ Features scaled:
  - Training shape: (11978, 5)
  - Test shape: (2995, 5)
  - Scaler expects 5 features

✅ VERIFIED: Scaler correctly expects exactly 5 features


## 6. Train Model


In [16]:
print("🤖 Training XGBoost Classifier...")

# Get number of classes
n_classes = len(np.unique(y_train))
print(f"  - Number of classes in training data: {n_classes}")
print(f"  - Class distribution in training:")
unique, counts = np.unique(y_train, return_counts=True)
for cls, count in zip(unique, counts):
    print(f"    Class {cls} ({label_encoder.inverse_transform([cls])[0]}): {count} samples")

# Validate we have enough classes
if n_classes < 2:
    raise ValueError(f"ERROR: Need at least 2 classes for classification, but found only {n_classes} class(es). "
                     f"All training samples have the same target category. Please check your data.")

# Train XGBoost model with proper multi-class configuration
if n_classes == 2:
    # Binary classification
    print("  - Using binary classification")
    model = xgb.XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        random_state=42,
        objective='binary:logistic',
        eval_metric='logloss',
        base_score=0.5,  # Valid base_score for binary classification
        use_label_encoder=False
    )
else:
    # Multi-class classification
    print(f"  - Using multi-class classification with {n_classes} classes")
    model = xgb.XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        random_state=42,
        objective='multi:softprob',
        eval_metric='mlogloss',
        num_class=n_classes,  # Explicitly set number of classes
        use_label_encoder=False
    )

model.fit(X_train_scaled, y_train)

print("✅ Model trained successfully!")

# Make predictions
y_pred = model.predict(X_test_scaled)
y_pred_proba = model.predict_proba(X_test_scaled)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"\n📊 Model Performance:")
print(f"  - Accuracy: {accuracy:.4f}")

# Classification report
print(f"\n📈 Classification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# Verify model expects 5 features
print(f"\n✅ Model Feature Verification:")
print(f"  - Model expects {model.n_features_in_} features")
assert model.n_features_in_ == 5, f"ERROR: Model expects {model.n_features_in_} features, but should be 5!"
print("✅ VERIFIED: Model correctly expects exactly 5 features")


🤖 Training XGBoost Classifier...
  - Number of classes in training data: 5
  - Class distribution in training:
    Class 0 (Dairy): 796 samples
    Class 1 (Fruits): 4397 samples
    Class 2 (Grains): 494 samples
    Class 3 (Spices): 1405 samples
    Class 4 (Vegetables): 4886 samples
  - Using multi-class classification with 5 classes
✅ Model trained successfully!

📊 Model Performance:
  - Accuracy: 0.9980

📈 Classification Report:
              precision    recall  f1-score   support

       Dairy       0.99      1.00      0.99       199
      Fruits       1.00      1.00      1.00      1100
      Grains       1.00      0.99      1.00       123
      Spices       1.00      0.99      0.99       351
  Vegetables       1.00      1.00      1.00      1222

    accuracy                           1.00      2995
   macro avg       1.00      1.00      1.00      2995
weighted avg       1.00      1.00      1.00      2995


✅ Model Feature Verification:
  - Model expects 5 features
✅ VERIFIED: M

## 7. Test Model with Sample Input


In [17]:
print("🧪 Testing model with sample input...")

# Sample input matching FastAPI request format
sample_input = {
    "totalOrders": 15,
    "purchaseFrequency": 3.2,
    "avgOrderValue": 2000.0,
    "lastPurchaseDaysAgo": 7,
    "totalItemsBought": 30
}

# Create DataFrame with exact feature order
sample_df = pd.DataFrame([sample_input])
sample_features = sample_df[FEATURE_COLUMNS].values

print(f"\n📥 Sample Input:")
for key, value in sample_input.items():
    print(f"  - {key}: {value}")

# Scale features
sample_scaled = scaler.transform(sample_features)

print(f"\n✅ Features scaled: shape {sample_scaled.shape}")
assert sample_scaled.shape[1] == 5, f"ERROR: Scaled features have {sample_scaled.shape[1]} columns, should be 5!"

# Make prediction
prediction = model.predict(sample_scaled)
prediction_proba = model.predict_proba(sample_scaled)

# Decode prediction
predicted_category = label_encoder.inverse_transform(prediction)[0]
prediction_probability = float(np.max(prediction_proba))

print(f"\n🎯 Prediction Result:")
print(f"  - Predicted Category: {predicted_category}")
print(f"  - Probability: {prediction_probability:.4f}")
print(f"  - Encoded: {prediction[0]}")

print(f"\n✅ Model test successful!")


🧪 Testing model with sample input...

📥 Sample Input:
  - totalOrders: 15
  - purchaseFrequency: 3.2
  - avgOrderValue: 2000.0
  - lastPurchaseDaysAgo: 7
  - totalItemsBought: 30

✅ Features scaled: shape (1, 5)

🎯 Prediction Result:
  - Predicted Category: Grains
  - Probability: 0.9612
  - Encoded: 2

✅ Model test successful!


In [18]:
print("💾 Saving model artifacts...")

# Save model
model_path = '../ml_models/customer_purchase_model.pkl'
scaler_path = '../ml_models/scaler.pkl'
encoder_path = '../ml_models/label_encoder.pkl'

# Try saving to ml_models directory
import os
os.makedirs('../ml_models', exist_ok=True)

joblib.dump(model, model_path)
print(f"✅ Model saved to: {model_path}")

joblib.dump(scaler, scaler_path)
print(f"✅ Scaler saved to: {scaler_path}")

joblib.dump(label_encoder, encoder_path)
print(f"✅ Label encoder saved to: {encoder_path}")

# Verify saved files
print(f"\n🔍 Verifying saved artifacts...")

# Load and verify model
loaded_model = joblib.load(model_path)
assert loaded_model.n_features_in_ == 5, f"ERROR: Loaded model expects {loaded_model.n_features_in_} features!"
print(f"✅ Model verified: expects {loaded_model.n_features_in_} features")

# Load and verify scaler
loaded_scaler = joblib.load(scaler_path)
assert loaded_scaler.n_features_in_ == 5, f"ERROR: Loaded scaler expects {loaded_scaler.n_features_in_} features!"
print(f"✅ Scaler verified: expects {loaded_scaler.n_features_in_} features")

# Test with sample input again using loaded model
test_pred = loaded_model.predict(loaded_scaler.transform(sample_features))
print(f"✅ Loaded model test prediction: {label_encoder.inverse_transform(test_pred)[0]}")

print(f"\n🎉 All artifacts saved and verified successfully!")
print(f"\n📋 Summary:")
print(f"  - Model: {model_path} (expects 5 features)")
print(f"  - Scaler: {scaler_path} (expects 5 features)")
print(f"  - Encoder: {encoder_path}")
print(f"  - Feature order: {FEATURE_COLUMNS}")


💾 Saving model artifacts...
✅ Model saved to: ../ml_models/customer_purchase_model.pkl
✅ Scaler saved to: ../ml_models/scaler.pkl
✅ Label encoder saved to: ../ml_models/label_encoder.pkl

🔍 Verifying saved artifacts...
✅ Model verified: expects 5 features
✅ Scaler verified: expects 5 features
✅ Loaded model test prediction: Grains

🎉 All artifacts saved and verified successfully!

📋 Summary:
  - Model: ../ml_models/customer_purchase_model.pkl (expects 5 features)
  - Scaler: ../ml_models/scaler.pkl (expects 5 features)
  - Encoder: ../ml_models/label_encoder.pkl
  - Feature order: ['totalOrders', 'purchaseFrequency', 'avgOrderValue', 'lastPurchaseDaysAgo', 'totalItemsBought']


## 9. Final Verification


In [19]:
print("🔍 Final Verification for FastAPI Compatibility...")

# Load all artifacts
final_model = joblib.load(model_path)
final_scaler = joblib.load(scaler_path)
final_encoder = joblib.load(encoder_path)

# Test with exact FastAPI input format
fastapi_input = np.array([[
    15.0,    # totalOrders
    3.2,     # purchaseFrequency
    2000.0,  # avgOrderValue
    7.0,     # lastPurchaseDaysAgo
    30.0     # totalItemsBought
]])

print(f"\n📥 FastAPI Input Shape: {fastapi_input.shape}")
print(f"  Expected: (1, 5)")

# Scale
scaled_input = final_scaler.transform(fastapi_input)
print(f"✅ Scaled Shape: {scaled_input.shape}")

# Predict
pred = final_model.predict(scaled_input)
proba = final_model.predict_proba(scaled_input)

# Decode
category = final_encoder.inverse_transform(pred)[0]
probability = float(np.max(proba))

print(f"\n🎯 Final Test Result:")
print(f"  {{")
print(f"    'predictedCategory': '{category}',")
print(f"    'predictionProbability': {probability:.4f},")
print(f"    'predictedCategoryEncoded': {pred[0]}")
print(f"  }}")

print(f"\n✅ ALL VERIFICATIONS PASSED!")
print(f"\n🚀 The model and scaler are ready for FastAPI deployment.")
print(f"   Both expect exactly 5 features in this order:")
for i, feat in enumerate(FEATURE_COLUMNS, 1):
    print(f"   {i}. {feat}")


🔍 Final Verification for FastAPI Compatibility...

📥 FastAPI Input Shape: (1, 5)
  Expected: (1, 5)
✅ Scaled Shape: (1, 5)

🎯 Final Test Result:
  {
    'predictedCategory': 'Grains',
    'predictionProbability': 0.9612,
    'predictedCategoryEncoded': 2
  }

✅ ALL VERIFICATIONS PASSED!

🚀 The model and scaler are ready for FastAPI deployment.
   Both expect exactly 5 features in this order:
   1. totalOrders
   2. purchaseFrequency
   3. avgOrderValue
   4. lastPurchaseDaysAgo
   5. totalItemsBought
